<a href="https://colab.research.google.com/github/defneerencan/defneerencan-Defne-Erencan-DSA210/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# GitHub raw CSV URL
url = "https://raw.githubusercontent.com/defneerencan/defneerencan-Defne-Erencan-DSA210/main/Expenses/akbank_clean_categorized_v3.csv"

df = pd.read_csv(url)
df.head()
